In [2]:
import os
import numpy as np
import glob
import pandas as pd  
import random
import math as mp
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import time

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow.keras.backend as K


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalMaxPooling1D

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from pyquaternion import Quaternion

from numpy import array
from numpy import hstack

from math import floor

In [11]:
def get_csv_files(dataset_path, keyword):
    """
    Generator function to recursively output the CSV files in a directory and its sub-directories.
    Arguments:
        dataset_path: Path to the directory containing the CSV files.
    Outputs:
        Paths of the found CSV files.
    """
    files = []
 
    for f in glob.glob(os.path.join(dataset_path,"*{}*".format(keyword))):
        #print(f)
        if not os.path.isdir(f):
            file_name, extension = f.split('.')            
            if extension == "csv":
                files.append(f)
            else:
                logging.warn("Invalid file: {}. Ignoring...".format(f))

    return files

In [12]:
def load_df(path): 
    df = pd.read_csv(path, index_col=None)

    return df

In [13]:
def split_sequences(sequences, n_steps, LAT, n_in_seq, n_out_seq):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        # find the end of this pattern
        stop_ix = i + n_steps
        shift_ix = i + n_steps + LAT
        seq_length = len(sequences) - LAT
        if stop_ix > seq_length:
          break
        seq_x = sequences[i:stop_ix,:-n_in_seq]
        seq_y = sequences[shift_ix-1, -n_out_seq:]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)
    #return X, y

In [40]:
def get_sequence_rotsonly(dataframe):
    # multivariate data preparation

    
    in_seq4 = dataframe[['qw']].to_numpy()
    in_seq5 = dataframe[['qx']].to_numpy()
    in_seq6 = dataframe[['qy']].to_numpy()
    in_seq7 = dataframe[['qz']].to_numpy()
    
    out_seq4 = dataframe[['qw']].to_numpy()
    out_seq5 = dataframe[['qx']].to_numpy()
    out_seq6 = dataframe[['qy']].to_numpy()
    out_seq7 = dataframe[['qz']].to_numpy()

    # convert to [rows, columns] structure
    
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    in_seq7 = in_seq7.reshape((len(in_seq7), 1))
 
    out_seq4 = out_seq4.reshape((len(out_seq4), 1))
    out_seq5 = out_seq5.reshape((len(out_seq5), 1))
    out_seq6 = out_seq6.reshape((len(out_seq6), 1))
    out_seq7 = out_seq7.reshape((len(out_seq7), 1))

    # horizontally stack columns
    dataset = hstack((in_seq4, in_seq5, in_seq6, in_seq7, out_seq4, out_seq5, out_seq6, out_seq7))
    
    #scaler_obj = scaler(dataset)
    #normalized_dataset = scaler_obj.transform(dataset)
    #print(normalized_dataset)
    
    # define number of input and output sequences
    n_in_seq = 4
    n_out_seq = 4
        
    # choose a number of time steps
    n_steps = 40 
    
    dt = 5 # sampling rate in ms
    LAT_in_ms = 100
    LAT_in_rows = 4 #4*16,66ms= 64ms

    # convert into input/output
    X, y = split_sequences(dataset, n_steps, LAT_in_rows, n_in_seq, n_out_seq)

    print("X shape is: " + str(X.shape))
    print("y shape is: " + str(y.shape))
    #print(type(X))
    print("------")
    
    return X,y

In [42]:
sample_quats_path = "C:/Users/weikert1/Documents/Thesis/Project_V04_12072021/Code/Checks/sample_quats.csv"
sample_quats = pd.read_csv(sample_quats_path, sep = ';')
sample_quats



,qw,qx,qy,qz
0,1,1,1,1
1,2,2,2,2
2,3,3,3,3
3,4,4,4,4
4,5,5,5,5
...,...,...,...,...
195,196,196,196,196
196,197,197,197,197
197,198,198,198,198
198,199,199,199,199


In [43]:
X, y = get_sequence_rotsonly(sample_quats)

X shape is: (157, 40, 4)
y shape is: (157, 4)
------


In [46]:
print(X[1])
print("-----------")
print(y[1])

[[ 2  2  2  2]
 [ 3  3  3  3]
 [ 4  4  4  4]
 [ 5  5  5  5]
 [ 6  6  6  6]
 [ 7  7  7  7]
 [ 8  8  8  8]
 [ 9  9  9  9]
 [10 10 10 10]
 [11 11 11 11]
 [12 12 12 12]
 [13 13 13 13]
 [14 14 14 14]
 [15 15 15 15]
 [16 16 16 16]
 [17 17 17 17]
 [18 18 18 18]
 [19 19 19 19]
 [20 20 20 20]
 [21 21 21 21]
 [22 22 22 22]
 [23 23 23 23]
 [24 24 24 24]
 [25 25 25 25]
 [26 26 26 26]
 [27 27 27 27]
 [28 28 28 28]
 [29 29 29 29]
 [30 30 30 30]
 [31 31 31 31]
 [32 32 32 32]
 [33 33 33 33]
 [34 34 34 34]
 [35 35 35 35]
 [36 36 36 36]
 [37 37 37 37]
 [38 38 38 38]
 [39 39 39 39]
 [40 40 40 40]
 [41 41 41 41]]
-----------
[45 45 45 45]


In [48]:
print(X[156])
print("-----------")
print(y[156])

[[157 157 157 157]
 [158 158 158 158]
 [159 159 159 159]
 [160 160 160 160]
 [161 161 161 161]
 [162 162 162 162]
 [163 163 163 163]
 [164 164 164 164]
 [165 165 165 165]
 [166 166 166 166]
 [167 167 167 167]
 [168 168 168 168]
 [169 169 169 169]
 [170 170 170 170]
 [171 171 171 171]
 [172 172 172 172]
 [173 173 173 173]
 [174 174 174 174]
 [175 175 175 175]
 [176 176 176 176]
 [177 177 177 177]
 [178 178 178 178]
 [179 179 179 179]
 [180 180 180 180]
 [181 181 181 181]
 [182 182 182 182]
 [183 183 183 183]
 [184 184 184 184]
 [185 185 185 185]
 [186 186 186 186]
 [187 187 187 187]
 [188 188 188 188]
 [189 189 189 189]
 [190 190 190 190]
 [191 191 191 191]
 [192 192 192 192]
 [193 193 193 193]
 [194 194 194 194]
 [195 195 195 195]
 [196 196 196 196]]
-----------
[200 200 200 200]


In [50]:
def cross_validation(X,y):
    
    tscv = TimeSeriesSplit(n_splits=3)
    for train_index, test_index in tscv.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

    print(X_train.shape) 
    print(X_test.shape) 
    print(y_train.shape) 
    print(y_test.shape)   
    
    return X_train, X_test, y_train, y_test

In [77]:

X_train_result = [] 
y_train_result = [] 

X_test_result = [] 
y_test_result = [] 
#X_result = np.array()

sample_quats_path = "C:/Users/weikert1/Documents/Thesis/Project_V04_12072021/Code/Checks/sample_quats.csv"
sample_quats = pd.read_csv(sample_quats_path, sep = ';')

X_single, y_single = get_sequence_rotsonly(sample_quats)
#X_train, X_test, y_train, y_test = train_test_split(X_single, y_single, test_size=0.20, random_state=42)
X_train, X_test, y_train, y_test = cross_validation(X_single,y_single)

X_train_result.append(X_train)
y_train_result.append(y_train)    
X_test_result.append(X_test)
y_test_result.append(y_test)

        


#-------------------------------------------------------


sample_quats_path_2 = "C:/Users/weikert1/Documents/Thesis/Project_V04_12072021/Code/Checks/sample_quats_2.csv"
sample_quats_2 = pd.read_csv(sample_quats_path_2, sep = ';')

X_single_2, y_single_2 = get_sequence_rotsonly(sample_quats_2)
X_train_2, X_test_2, y_train_2, y_test_2 = cross_validation(X_single_2,y_single_2)



X_train_result.append(X_train_2)
y_train_result.append(y_train_2)
X_test_result.append(X_test_2)
y_test_result.append(y_test_2)


#-------------------------------------------------------


X_train_result = np.vstack((X_train_result))
y_train_result = np.vstack((y_train_result))

X_test_result = np.vstack((X_test_result))
y_test_result = np.vstack((y_test_result))
        


X shape is: (157, 40, 4)
y shape is: (157, 4)
------
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] TEST: [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63
 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78] TEST: [ 79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96
  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114
 115 116 117]
TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  

In [90]:
X_train_result.shape

X_test_result.shape

(78, 40, 4)

In [111]:
X_train_result[38:50]


array([[[39, 39, 39, 39],
        [40, 40, 40, 40],
        [41, 41, 41, 41],
        ...,
        [76, 76, 76, 76],
        [77, 77, 77, 77],
        [78, 78, 78, 78]],

       [[40, 40, 40, 40],
        [41, 41, 41, 41],
        [42, 42, 42, 42],
        ...,
        [77, 77, 77, 77],
        [78, 78, 78, 78],
        [79, 79, 79, 79]],

       [[41, 41, 41, 41],
        [42, 42, 42, 42],
        [43, 43, 43, 43],
        ...,
        [78, 78, 78, 78],
        [79, 79, 79, 79],
        [80, 80, 80, 80]],

       ...,

       [[48, 48, 48, 48],
        [49, 49, 49, 49],
        [50, 50, 50, 50],
        ...,
        [85, 85, 85, 85],
        [86, 86, 86, 86],
        [87, 87, 87, 87]],

       [[49, 49, 49, 49],
        [50, 50, 50, 50],
        [51, 51, 51, 51],
        ...,
        [86, 86, 86, 86],
        [87, 87, 87, 87],
        [88, 88, 88, 88]],

       [[50, 50, 50, 50],
        [51, 51, 51, 51],
        [52, 52, 52, 52],
        ...,
        [87, 87, 87, 87],
        [88, 88

In [97]:
y_train_result[1]

array([45, 45, 45, 45], dtype=int64)

In [98]:
X_test_result[1]

array([[120, 120, 120, 120],
       [121, 121, 121, 121],
       [122, 122, 122, 122],
       [123, 123, 123, 123],
       [124, 124, 124, 124],
       [125, 125, 125, 125],
       [126, 126, 126, 126],
       [127, 127, 127, 127],
       [128, 128, 128, 128],
       [129, 129, 129, 129],
       [130, 130, 130, 130],
       [131, 131, 131, 131],
       [132, 132, 132, 132],
       [133, 133, 133, 133],
       [134, 134, 134, 134],
       [135, 135, 135, 135],
       [136, 136, 136, 136],
       [137, 137, 137, 137],
       [138, 138, 138, 138],
       [139, 139, 139, 139],
       [140, 140, 140, 140],
       [141, 141, 141, 141],
       [142, 142, 142, 142],
       [143, 143, 143, 143],
       [144, 144, 144, 144],
       [145, 145, 145, 145],
       [146, 146, 146, 146],
       [147, 147, 147, 147],
       [148, 148, 148, 148],
       [149, 149, 149, 149],
       [150, 150, 150, 150],
       [151, 151, 151, 151],
       [152, 152, 152, 152],
       [153, 153, 153, 153],
       [154, 1

In [99]:
y_test_result[1]

array([163, 163, 163, 163], dtype=int64)

In [157]:
path_test = "C:/Users/weikert1/Documents/Thesis/Project_V04_12072021/Code/Checks/sample_quats.csv"
result_path = "C:/Users/weikert1/Documents/Thesis/Project_V04_12072021/Code/Checks/"

n_features = 4
n_steps = 40
LAT_in_rows = 4
preds_rot = []
true_rot = []

csv_df = pd.read_csv(path_test, sep = ';')
in_seq4 = csv_df[['qw']].to_numpy()
in_seq5 = csv_df[['qx']].to_numpy()
in_seq6 = csv_df[['qy']].to_numpy()
in_seq7 = csv_df[['qz']].to_numpy()
test_data = array(hstack((in_seq4, in_seq5, in_seq6, in_seq7)))

print("Length of test data is " + str(len(test_data)))

n_preds =  len(test_data) - (n_steps + LAT_in_rows) 
print("N_preds is " + str(n_preds))

for i in range(n_preds):
#for i in range(100):

    x_input = test_data[i:i+n_steps]


    if i == 0:
        print(x_input)

    if i == n_preds-1:
        print(x_input)


    # Reshape x_input
    #x_input = x_input.reshape((1, n_steps, n_features))
    yhat = test_data[i+n_steps+LAT_in_rows]
    ytrue = test_data[i+n_steps+LAT_in_rows]

    preds_rot.append(yhat)
    true_rot.append(ytrue)

    
preds_rot = array(preds_rot)
true_rot = array(true_rot)

############ Save prediction per timestamp (preds_rot)
df_preds_rot = pd.DataFrame(np.row_stack(preds_rot), columns=['qw_pred','qx_pred','qy_pred','qz_pred'])
df_true_rot = pd.DataFrame(np.row_stack(true_rot), columns=['qw_true','qx_true','qy_true','qz_true'])
df_rot_result = pd.concat([df_preds_rot, df_true_rot], axis=1)

#df_preds_rot_and_eulers = add_eulers(df_preds_rot)

df_rot_result.to_csv(os.path.join(result_path + '_preds_rot_lstm.csv'), index=False)

#print("Single Predictions are saved")

#print("All the predictions are saved to result path")

Length of test data is 200
N_preds is 156
[[ 1  1  1  1]
 [ 2  2  2  2]
 [ 3  3  3  3]
 [ 4  4  4  4]
 [ 5  5  5  5]
 [ 6  6  6  6]
 [ 7  7  7  7]
 [ 8  8  8  8]
 [ 9  9  9  9]
 [10 10 10 10]
 [11 11 11 11]
 [12 12 12 12]
 [13 13 13 13]
 [14 14 14 14]
 [15 15 15 15]
 [16 16 16 16]
 [17 17 17 17]
 [18 18 18 18]
 [19 19 19 19]
 [20 20 20 20]
 [21 21 21 21]
 [22 22 22 22]
 [23 23 23 23]
 [24 24 24 24]
 [25 25 25 25]
 [26 26 26 26]
 [27 27 27 27]
 [28 28 28 28]
 [29 29 29 29]
 [30 30 30 30]
 [31 31 31 31]
 [32 32 32 32]
 [33 33 33 33]
 [34 34 34 34]
 [35 35 35 35]
 [36 36 36 36]
 [37 37 37 37]
 [38 38 38 38]
 [39 39 39 39]
 [40 40 40 40]]
[[156 156 156 156]
 [157 157 157 157]
 [158 158 158 158]
 [159 159 159 159]
 [160 160 160 160]
 [161 161 161 161]
 [162 162 162 162]
 [163 163 163 163]
 [164 164 164 164]
 [165 165 165 165]
 [166 166 166 166]
 [167 167 167 167]
 [168 168 168 168]
 [169 169 169 169]
 [170 170 170 170]
 [171 171 171 171]
 [172 172 172 172]
 [173 173 173 173]
 [174 174 174 1

In [155]:
df_preds_rot = pd.DataFrame(np.row_stack(preds_rot), columns=['qw_pred','qx_pred','qy_pred','qz_pred'])
df_true_rot = pd.DataFrame(np.row_stack(true_rot), columns=['qw_true','qx_true','qy_true','qz_true'])
df_rot_result = pd.concat([df_preds_rot, df_true_rot], axis=1)


In [156]:
df_rot_result

,qw_pred,qx_pred,qy_pred,qz_pred,qw_true,qx_true,qy_true,qz_true
0,45,45,45,45,45,45,45,45
1,46,46,46,46,46,46,46,46
2,47,47,47,47,47,47,47,47
3,48,48,48,48,48,48,48,48
4,49,49,49,49,49,49,49,49
...,...,...,...,...,...,...,...,...
151,196,196,196,196,196,196,196,196
152,197,197,197,197,197,197,197,197
153,198,198,198,198,198,198,198,198
154,199,199,199,199,199,199,199,199


In [150]:
preds_rot

array([[ 45,  45,  45,  45],
       [ 46,  46,  46,  46],
       [ 47,  47,  47,  47],
       [ 48,  48,  48,  48],
       [ 49,  49,  49,  49],
       [ 50,  50,  50,  50],
       [ 51,  51,  51,  51],
       [ 52,  52,  52,  52],
       [ 53,  53,  53,  53],
       [ 54,  54,  54,  54],
       [ 55,  55,  55,  55],
       [ 56,  56,  56,  56],
       [ 57,  57,  57,  57],
       [ 58,  58,  58,  58],
       [ 59,  59,  59,  59],
       [ 60,  60,  60,  60],
       [ 61,  61,  61,  61],
       [ 62,  62,  62,  62],
       [ 63,  63,  63,  63],
       [ 64,  64,  64,  64],
       [ 65,  65,  65,  65],
       [ 66,  66,  66,  66],
       [ 67,  67,  67,  67],
       [ 68,  68,  68,  68],
       [ 69,  69,  69,  69],
       [ 70,  70,  70,  70],
       [ 71,  71,  71,  71],
       [ 72,  72,  72,  72],
       [ 73,  73,  73,  73],
       [ 74,  74,  74,  74],
       [ 75,  75,  75,  75],
       [ 76,  76,  76,  76],
       [ 77,  77,  77,  77],
       [ 78,  78,  78,  78],
       [ 79,  